# Laboratorio 09

Ejecutar el siguiente comando para descargar los recursos necesarios para usar en el laboratorio:

In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

En la ventana que se abrirá, debe descargar por lo menos los siguientes recursos: <b>
<br> Corpora: stopwords
<img src="NLTK_Corpora.png" width="700">
<br> Corpora: opinion_lexicon
<img src="NLTK_Lexicon.png" width="700">
<br> Models: averaged_perceptron_tagger
<br> Models: punkt
<img src="NLTK_Models.png" width="700">
<br> Models: universal_tagset
<img src="NLTK_Tagset.png" width="700">

## Dataset:
(Se trabajará con el dataset de Movie Reviews visto en clase)

In [2]:
#Archivo con contenido de reviews de películas
g = open('reviews.txt','r') 
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

#Archivo con las etiquetas (POSITIVE,NEGATIVE)
g = open('labels.txt','r') 
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

In [3]:
from collections import Counter
import numpy as np

<b> Se va a trabajar sólo con 1000 reviews para no ralentizar la tarea solicitada:

In [4]:
reviews = reviews[:1000]
labels = labels[:1000]

## P1. POS-Tagger (8 ptos)

Fuente: http://www.nltk.org/book/ch05.html

In [5]:
import nltk
from nltk import word_tokenize

In [6]:
#Ejemplo de cómo usar el POS-tagger de NTLT con el tagset universal:
text = word_tokenize("And now for something different")
POStags = nltk.pos_tag(text, tagset="universal")
print POStags
print POStags[0]
print POStags[0][1]
print POStags[0][1]=="CONJ"

[('And', u'CONJ'), ('now', u'ADV'), ('for', u'ADP'), ('something', u'NOUN'), ('different', u'ADJ')]
('And', u'CONJ')
CONJ
True


### P1.1. Obtener el POS-Tag de todo el dataset (2 ptos)
Guarde una variable "reviews_POStags" que contenga todos los reviews con las palabras divididas y etiquetadas con su POS-Tag

In [7]:
reviews_POStags = []
count = 0
for r in reviews:
    if (count % 100 == 0): 
        print count
    count +=1
    text = word_tokenize(r)
    ptags = nltk.pos_tag(text, tagset="universal")
    reviews_POStags.append(ptags)

0
100
200
300
400
500
600
700
800
900


In [8]:
reviews_POStags[0]

[('bromwell', u'ADV'),
 ('high', u'ADJ'),
 ('is', u'VERB'),
 ('a', u'DET'),
 ('cartoon', u'NOUN'),
 ('comedy', u'NOUN'),
 ('.', u'.'),
 ('it', u'PRON'),
 ('ran', u'VERB'),
 ('at', u'ADP'),
 ('the', u'DET'),
 ('same', u'ADJ'),
 ('time', u'NOUN'),
 ('as', u'ADP'),
 ('some', u'DET'),
 ('other', u'ADJ'),
 ('programs', u'NOUN'),
 ('about', u'ADP'),
 ('school', u'NOUN'),
 ('life', u'NOUN'),
 ('such', u'ADJ'),
 ('as', u'ADP'),
 ('teachers', u'NOUN'),
 ('.', u'.'),
 ('my', u'PRON'),
 ('years', u'NOUN'),
 ('in', u'ADP'),
 ('the', u'DET'),
 ('teaching', u'NOUN'),
 ('profession', u'NOUN'),
 ('lead', u'VERB'),
 ('me', u'PRON'),
 ('to', u'PRT'),
 ('believe', u'VERB'),
 ('that', u'ADP'),
 ('bromwell', u'ADV'),
 ('high', u'ADJ'),
 ('s', u'NOUN'),
 ('satire', u'NOUN'),
 ('is', u'VERB'),
 ('much', u'ADV'),
 ('closer', u'ADV'),
 ('to', u'PRT'),
 ('reality', u'NOUN'),
 ('than', u'ADP'),
 ('is', u'VERB'),
 ('teachers', u'NOUN'),
 ('.', u'.'),
 ('the', u'DET'),
 ('scramble', u'ADJ'),
 ('to', u'PRT'),
 ('su

### P1.2. Filtrar los términos que corresponden a una categoría específica (2 ptos)
A partir de la variable "reviews_POStags", filtre todos los términos que sean de una categoría en específico, como el <b>ADJ</b> (Adjetivo).

In [11]:
#Filtre una lista de solo "ADJ"
ADJ_tokens = []
for review in reviews_POStags:
    for term in review:
        if term[1]=='ADJ':
            ADJ_tokens.append(term[0])
print len(ADJ_tokens)

25450


### P1.3. Construir frases a partir de las categorías identificadas (4 ptos)
A partir de la variable "reviews_POStags" extraiga todas las frases del dataset que cumplan un patrón determinado, en este caso: <b>NOUN+NOUN</b> (2 sustantivos seguidos)

In [18]:
#Identifique y guarde todas las frases del tipo NOUN+NOUN en el dataset
NN_tokens = []
i = 0
for review in reviews_POStags:
    for i in range(len(review)-1):
        if review[i][1]=='NOUN' and review[i+1][1]=='NOUN':
            NN_tokens.append(review[i][0]+' '+review[i+1][0])
print len(NN_tokens)
print NN_tokens[0]
print NN_tokens[50]

11947
cartoon comedy
right bang


## P2. Lexicon (8 ptos)

### P2.1. (3 puntos)
<b>Construya un Lexicon de términos POS y NEG de sólo ADJ (adjetivos) usando el "factor de polaridad" mostrado en el ejercicio de clase. Debe obtener una estructura del tipo diccionario como la siguiente: 

In [19]:
lexicon = {}
lexicon["term_POS"] = 1.5
lexicon["term_NEG"] = -1.5
print lexicon

{'term_POS': 1.5, 'term_NEG': -1.5}


In [34]:
#Cálculo del factor de polaridad:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

for review, label in zip(reviews, labels):
    if label == 'POSITIVE':
        positive_counts.update(set(review.split(" ")))
    else:
        negative_counts.update(set(review.split(" ")))
total_counts.update(positive_counts)
total_counts.update(negative_counts)

frequency_cutoff = 50
pos_neg_ratios = Counter()
for term, cnt in list(total_counts.most_common()):
    if cnt >= frequency_cutoff: # common words only
        pos_neg_ratios[term] = np.log((positive_counts[term]+0.0001) / \
                                      (negative_counts[term]+1))
        
polarity_cutoff = 0.9
review_vocab={}
for x in pos_neg_ratios.keys():
    if pos_neg_ratios[x] >= polarity_cutoff or pos_neg_ratios[x] <= -polarity_cutoff:
        review_vocab[x]=pos_neg_ratios[x]

#review_vocab = [x for x in pos_neg_ratios.keys() \
#                if pos_neg_ratios[x] >= polarity_cutoff or \
#                   pos_neg_ratios[x] <= -polarity_cutoff]

print review_vocab

{'beautiful': 1.2878560124430829, 'heart': 0.98861371903247242, 'based': 0.94098578348594253, 'couldn': -1.4109769737602618, 'money': -0.92494444680063437, 'boring': -1.4954859608067759, 'bad': -0.99425092199342857, 'overall': 0.91629295409390821, 'terrible': -1.5040673968262737, 'budget': -0.98860514347331241, 'nothing': -1.0573662998356215, 'amazing': 1.5339323207083468, 'wonderful': 1.3470756479646093, 'worst': -2.28999381086531, 'excellent': 0.98083103872441757, 'awful': -1.8827169618215358, 'poor': -1.1192253258903766, 'perfect': 1.4069156891368704, 'waste': -2.871646292106222, 'true': 1.0541620412476431}


<b>Para filtrar los términos y armar el lexicon, puede aprovechar el listado calculado en la Pregunta P1.2:

In [49]:
newADJ_tokens={}
for word in review_vocab.keys():
    if (word in ADJ_tokens):
        newADJ_tokens[word]=review_vocab[word]
print newADJ_tokens

{'beautiful': 1.2878560124430829, 'poor': -1.1192253258903766, 'couldn': -1.4109769737602618, 'boring': -1.4954859608067759, 'overall': 0.91629295409390821, 'terrible': -1.5040673968262737, 'amazing': 1.5339323207083468, 'bad': -0.99425092199342857, 'wonderful': 1.3470756479646093, 'worst': -2.28999381086531, 'excellent': 0.98083103872441757, 'awful': -1.8827169618215358, 'perfect': 1.4069156891368704, 'true': 1.0541620412476431}


### P2.2. (5 puntos)
<b> Se quiere extender el Lexicon para que también pueda guardar valores de polaridad de algunas entidades. En este sentido, se añadirán NOUN que puedan identificarse en patrones del tipo: NOUN+ADJ o ADJ+NOUN, donde:
<br> - La palabra ADJ ya debe existir en el Lexicon (según la pregunta anterior)
<br> - El valor de polaridad del NOUN a añadir será el mismo que el que tenga el ADJ en el Lexicon
<br> - En caso de que un término NOUN acompañe a más de un ADJ existente a la vez, puede optar por elegir uno al azar

In [50]:
#Armar todos los NOUNS que cumplen el patron en una lista de tuplas (NOUN,ADJ)
NN_tokens = []
i = 0
for review in reviews_POStags:
    for i in range(len(review)):
        if review[i][1]=='NOUN' and review[i-1][1]=='ADJ':
            NN_tokens.append((review[i][0],review[i-1][0]))
        if i < (len(review)-1):
            if review[i][1]=='NOUN' and review[i+1][1]=='ADJ':
                NN_tokens.append((review[i][0],review[i+1][0]))
print len(NN_tokens)
print NN_tokens[0]

#Iterar a través de todos los ADJs en nuestro Lexicon:
for word in NN_tokens:
    if (word[1] in newADJ_tokens.keys()):
        print word[1]
        newADJ_tokens[word[0]]=newADJ_tokens[word[1]]
print len(newADJ_tokens)

16752
('time', 'same')
overall
boring
worst
true
true
true
excellent
true
true
poor
true
excellent
bad
worst
worst
bad
bad
excellent
boring
poor
excellent
boring
awful
boring
bad
worst
true
perfect
i
excellent
i
i
i
bad
bad
i
overall
poor
poor
boring
worst
i
worst
i
bollywood
beautiful
wonderful
terrible
i
i
worst
bad
bad
awful
terrible
true
i
i
i
br
worst
s
s
i
i
overall
br
i
overall
s
poor
overall
overall
overall
overall
overall
overall
overall
overall
overall
overall
br
s
worst
don
br
bad
bad
amazing
amazing
wonderful
true
excellent
boring
bad
boring
awful
br
overall
amazing
br
t
br
overall
i
amazing
br
br
br
overall
t
t
beautiful
terrible
terrible
bad
amazing
titanic
br
i
overall
titanic
amazing
beautiful
beautiful
br
worst
br
saif
kareena
t
amazing
br
s
s
bad
br
br
br
s
titanic
titanic
titanic
s
s
engagement
beautiful
s
br
s
amazing
amazing
kareena
bad
bad
bad
terrible
kareena
bad
s
t
saif
s
visual
br
br
br
br
br
br
br
br
s
s
br
titanic
titanic
br
s
visual
s
bad
br
akshay
akshay
b

## P3. Negación (4 ptos)
Si analiza los siguientes análisis de polaridad basados en sólo un lexicon, se observa que buscar la coincidencia de un término en un diccionario puede resultar inefectiva ante la presencia de una negación:

In [51]:
from nltk import sentiment
sentence_1 = "I like the camera."
print sentence_1
nltk.sentiment.util.demo_liu_hu_lexicon(sentence_1, plot=False)

I like the camera.
Positive


/opt/anaconda2/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [52]:
sentence_2 = "I hate the camera."
print sentence_2
nltk.sentiment.util.demo_liu_hu_lexicon(sentence_2, plot=False)

I hate the camera.
Negative


In [53]:
sentence_3 = "I don't like the camera."
print sentence_3
nltk.sentiment.util.demo_liu_hu_lexicon(sentence_3, plot=False)

I don't like the camera.
Positive


<b>En este sentido, se solicita implementar un método que, al identificar un término de negación (didn't haven't don't aren't isn't ...) modifique los términos que siguen en el texto hasta encontrar una puntuación (CATEGORIA: '.').
<br>Ejecute dicho procesamiento sobre todo el dataset y transforme los reviews

In [54]:
NEGATE = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

text = word_tokenize("I don't like the camera.")
nltk.pos_tag(text, tagset="universal")

[('I', u'PRON'),
 ('do', u'VERB'),
 ("n't", u'ADV'),
 ('like', u'VERB'),
 ('the', u'DET'),
 ('camera', u'NOUN'),
 ('.', u'.')]

In [59]:
import string
type (string.punctuation)

str

In [64]:
def transform_negation(sentence):
    #Split
    #Analizar cada Token
    for i in range(len(sentence)):
        #Si el token es un didn't u otra cosa
        if sentence[i] in NEGATE:
            #Iterar desde la siguiente posicion hasta el final
            for j in range(i+1,len(sentence)):
                #Pero en verdad hasta el siguiente signo de puntuacion
                if (sentence[j] in string.punctuation):
                    break
                #Modificar la siguiente parte
                sentence[j]='NOT_'+sentence[j]
                #Lo ideal seria saltarnos todas las palabras modificadas, haciendo un i=j, pero no es necesario

    sentence_NEGATED = ' '.join(sentence)            
    return sentence_NEGATED

In [69]:
#PRUEBA
text = "I don't like the camera."
sentence = text.split()
print transform_negation(sentence)
print sentence

I don't NOT_like NOT_the NOT_camera.
['I', "don't", 'NOT_like', 'NOT_the', 'NOT_camera.']


In [77]:
#CONVERSION DE TODO EL DATASET
NEWreviews=[]
print len(reviews)
for review in reviews:
    sentence = review.split()
    NEWreviews.append(transform_negation(sentence))

1000


In [79]:
#Prueba
print reviews[50]
print NEWreviews[50]

aro tolbukhin burnt alive seven people in a mission in guatemala in the    s . also he declared that he had murdered another   people  he used to kill pregnant women  and then he set them on fire  .  br    br   this movie is a documentary that portraits the personality of aro through several interviews with people that got to know him and through some scenes played by actors based on real facts .  br    br    aro tolbukhin  is a serious work  so analytical  it  s not morbid at all . such a horrifying testimony about how some childhood trauma can turn a man into a monster .  br    br   my rate      
aro tolbukhin burnt alive seven people in a mission in guatemala in the s . also he declared that he had murdered another people he used to kill pregnant women and then he set them on fire . br br this movie is a documentary that portraits the personality of aro through several interviews with people that got to know him and through some scenes played by actors based on real facts . br br ar